# COBRApy

COBRApy is a package for constraint-based modeling of biological networks written in Python.

This tool allows loading and inspecting Genome-Scale Metabolic (GEM) models written in the Systems Biology Markup Language (SBML) format.

Using COBRApy, one can analyze the following model contents:
1. Reactions
2. Metabolites
3. Genes
4. Exchange reactions (Environmental Conditions)

COBRApy allows manipulating the contents of a GEM model. For instance, one can edit reactions' flux bounds, knock out a metabolic gene, or change environmental conditions.

COBRApy contains flux analysis methods to simulate an organism's phenotypic behavior. These include Flux Balance Analysis (FBA), Parsimonious FBA, or Flux Variability Analysis (FVA).

The simulation of gene and reaction deletions for a given GEM model is a straightforward process. One can simulate single or double knockouts using one of the flux analysis methods mentioned above.

## Installation


### Requirements
The following requirements are needed to use COBRApy:
- Python 3.6 or higher
- pip must be installed
- GLPK is the default solver, but CPLEX is preferred


### How to install COBRApy?
```
pip install cobra
```

# Exercise 2 - Phenotype prediction

## Working with a GEM model

For this practical session, we will be using the following model:
- _E. coli_ core model which contains the central carbon metabolism of _Escherichia coli_ -> file: ../data/e_coli_core.xml

You can read more about _E. coli_ core model (Orth et al., 2010) in the following links:
- https://journals.asm.org/doi/10.1128/ecosalplus.10.2.1
- http://bigg.ucsd.edu/models/e_coli_core

This exercise consists of exploring the phenotype prediction tools of COBRApy. Thus, the following steps will be followed:
- Perform an FBA simulation using an aerobic/anaerobic conditions;
- Perform an pFBA simulation;
- Perform an FVA simulation;

In [1]:
# imports
import cobra
import escher

In [2]:
# Loading a model
model_path = '../data/e_coli_core.xml'
model = cobra.io.read_sbml_model(model_path)

model

## Phenotype Prediction

COBRApy includes different algorithms for phenotype prediction. These include:
- Flux Balance Analysis (FBA); 
- Parsimonious Flux Balance Analysis (pFBA);
- Flux Variability Analysis (FVA);
- Regulatory On/Off Minimization (ROOM);
- Minimization of Metabolic Adjustment (MOMA);

### Flux Balance Analysis (FBA) - Aerobiosis

First, the exchange reactions should be verified to make sure that the aerobic conditions are all set up in the model.

In [3]:
#inspecting the exchange reactions.
for exchange in model.exchanges:
    print(exchange.name, '->', exchange.bounds)

In [4]:
# performing a FBA simulation in aerobiosis
model.summary()

### Flux Balance Analysis (FBA) - Anaerobiosis

Now, we should alter the exchange reactions to anaerobic conditions.

In [5]:
#setting an anaerobic medium
o2_exchange = model.exchanges.get_by_id('EX_o2_e')
o2_exchange.bounds = (0, 1000)
o2_exchange

In [6]:
# performing a FBA simulation in anaerobiosis
model.summary()

In [7]:
# now we should revert the model changes to aerobic conditions
o2_exchange.bounds = (-1000, 1000)
o2_exchange

### Making reversible changes in the model

All changes performed in a GEM model using CORBApy are irreversible by default.
That is, if we change the bounds of the oxygen exchange reaction, our model will no longer continue under aerobic conditions during this exercise.

However, there is a way to perform reversible changes in a GEM model using COBRApy. For that, one can use the `with` context manager in our `model`. All changes performed within the `with` context manager block will be reverted automatically by COBRApy.

In [8]:
# verifying that we have reverted the model changes to aerobic conditions
o2_exchange = model.exchanges.get_by_id('EX_o2_e')
o2_exchange

In [9]:
# making reversible changes in the model
with model:
    o2_exchange.bounds = (0, 1000)
    model_summary = model.summary()

model_summary

In [10]:
# the bounds of the oxygen exchange reaction have been reverted automatically this time
o2_exchange

### Flux Variability Analysis (FVA)

FBA can only obtain a unique flux distribution for a given objective function. Nevertheless, the space of flux distributions is very large and can vary significantly for a different objective. _FVA_ is a simulation method that finds the possible flux range for each reaction. _FVA_ can be used from the flux analysis package `cobra.flux_analysis.flux_variability_analysis(model)`. Note that, _FVA_ allows setting a minimum value of growth rate. In this case, we will be using 10% (that is, 0.1) of the wild-type growth rate.

In [11]:
# performing fva simulation
fva_solution = cobra.flux_analysis.flux_variability_analysis(model, fraction_of_optimum=0.1)
fva_solution

In [12]:
# maximum theoretical production rates of Acetate (EX_ac_e), Ethanol (EX_etoh_e), and Formate (EX_for_e)

print(f'Maximum theoretical production rate of Acetate:', fva_solution.loc['EX_ac_e', 'maximum'], 'mmol/gDW/h')
print(f'Maximum theoretical production rate of Ethanol:', fva_solution.loc['EX_etoh_e', 'maximum'], 'mmol/gDW/h')
print(f'Maximum theoretical production rate of Formate:', fva_solution.loc['EX_for_e', 'maximum'], 'mmol/gDW/h')

### Parsimonious Flux Balance Analysis (pFBA)

pFBA simulations gives the optimal growth rate, while minimizing the total sum of fluxes.
pFBA can be used from the flux analysis package `cobra.flux_analysis.pfba(model)`.

In [13]:
#performing pfba simulation
pfba_solution = cobra.flux_analysis.pfba(model)
pfba_solution

The optimal solution of the pFBA is considerably different from the FBA result. This happens because the objective value for the pFBA is defined as the sum of all flux values (`sum(abs(pfba_solution.fluxes.values))`). On the other hand, the FBA result corresponds to the flux value of the reaction that is being optimized (`fba_solution.fluxes["BIOMASS_Ecoli_core_w_GAM"]`).

In [14]:
#calculating the objective value of a pFBA solution
sum(abs(pfba_solution.fluxes.values))